<a href="https://colab.research.google.com/github/JesseRinzel123/SSL-ASR-Cross-Talker-Generalization/blob/main/glmm_plot_Bradlow23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq && apt-get install -y -qq r-cran-lme4


In [63]:
#You need these code to allow colab access your google cloud space.
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/ASR-Cross-Talker-Generalization')

#It will bring up a new screen. Just follow the instructions.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import copy
import tqdm
import numpy as np
import os
!pip install textgrid
import textgrid
import itertools
import multiprocessing
import time
from joblib import Parallel, delayed
import pickle
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula, pandas2ri

import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

### load audio file path

In [65]:
def get_pathset(paths):
    return [os.path.join(dir, each_file) for dir, mid, files in os.walk(paths) for each_file in files if each_file.endswith(".wav")]
audio_dir ="data/speech_Bradlow"

### load human data from xlsx file

In [66]:
human_result_path="data/BBP-2023-TestData2.xlsx"
human_result = pd.read_excel(human_result_path)
human_result=human_result[human_result["training_condition"]!="a_control"]


### load 3d-tSNE representations from pkl file

In [67]:
with open("data/hubert_T24.pkl", "rb") as file:
    tSNE_representations = pickle.load(file)

In [68]:
from numba import njit
@njit
def weighted_minkowski(vec1, vec2,  tau, w=1):

    total = 0.0
    for m in range(len(vec1)):
        diff = w*abs(vec1[m] - vec2[m])
        total += (diff ** tau)
    return total**(1/tau)#np.sqrt(total)

@njit
def dtw_sim(seq1, seq2,  tau, k):
    n, m = len(seq1), len(seq2)
    dtw_matrix = np.full((n+1, m+1), np.inf)
    dtw_matrix[0, 0] = 0.0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = weighted_minkowski(seq1[i-1], seq2[j-1], tau)
            dtw_matrix[i, j] = cost + min(dtw_matrix[i-1, j],    # insertion
                                         dtw_matrix[i, j-1],    # deletion
                                         dtw_matrix[i-1, j-1])  # match
    return np.exp(-(dtw_matrix[n, m]/((n+m)/2))*k)# change to *k,


def create_distance_matrix(sentence_matrix,tau, k):
    distance_matirx=np.zeros((60,4,4))
    for _,i in enumerate(sentence_matrix):
        for _1,each_talker1 in enumerate(i):
            for _2,each_talker2 in enumerate(i):
                distance_matirx[_][_1][_2]=dtw_sim(each_talker1, each_talker2, tau, k)
    return distance_matirx


In [69]:
def get_sentence_ind(audio_path,human_result):
    tg = textgrid.TextGrid.fromFile(audio_path[:-3]+"TextGrid")
    tg_sentence = [i for i in tg[0] if i.mark!=""]
    hint1=[]
    for _,i in enumerate(tg_sentence):
        if i.mark in set(human_result['sentence_test']):
            hint1.append(_)
    return hint1


def sim_measure(df, distance_matirx,tg_sentence_list,talkers):
    '''
    This function is for computing the similarity values for each row of human data.
    '''

    out_df=pd.DataFrame(columns=['Condition', 'TrainingTalker', 'TestTalker','ListenerID' ,'Sentence',  'similarity', 'score_test_logit','numCorrect','numWord'])
    for each_ in tqdm.tqdm(df.values):
        if each_[1][0]=="b":
            ind_T = talkers.index(each_[1][-3:])
            arr = distance_matirx[tg_sentence_list.index(each_[df.columns.get_loc("sentence_test")])][ind_T]
            ind_test=talkers.index(each_[-3])
            out_df.loc[len(out_df)]=[each_[df.columns.get_loc("st_mt_cnt")],
                                        each_[df.columns.get_loc("training_condition")],
                                        each_[df.columns.get_loc("speaker_test")],
                                        str(each_[df.columns.get_loc("id2")]),
                                        each_[df.columns.get_loc("sentence_test")],
                                        arr[ind_test],
                                        each_[df.columns.get_loc("score_test_logit")],
                                        each_[df.columns.get_loc("score_test")],
                                        each_[df.columns.get_loc("possible_score_test")]
                                        ]
        else:
            ind_T = talkers.index(each_[1][-3:])
            ind_test = talkers.index(each_[-3])
            arr = distance_matirx[tg_sentence_list.index(each_[df.columns.get_loc("sentence_test")])][ind_T]
            #indices=np.nonzero(arr)
            arr_non_1 = arr[arr!=1]
            if each_[1][-3:] == each_[-3]:
                out_df.loc[len(out_df)]=[each_[df.columns.get_loc("st_mt_cnt")],
                                         each_[df.columns.get_loc("training_condition")],
                                         each_[df.columns.get_loc("speaker_test")],
                                         str(each_[df.columns.get_loc("id2")]),
                                         each_[df.columns.get_loc("sentence_test")],
                                         np.mean(arr_non_1),
                                         each_[df.columns.get_loc("score_test_logit")],
                                         each_[df.columns.get_loc("score_test")],
                                         each_[df.columns.get_loc("possible_score_test")]
                                         ]
    return out_df



In [70]:
audio_path=get_pathset(audio_dir)
tg = textgrid.TextGrid.fromFile(audio_path[0][:-3]+"TextGrid")
tg_sentence = [i for i in tg[0] if i.mark!=""]
hint1_list=get_sentence_ind(audio_path[0],human_result)
tg_sentence=[tg_sentence[i].mark for i in hint1_list]
tg1 = textgrid.TextGrid.fromFile(audio_path[1][:-3]+"TextGrid")
tg_sentence1 = [i for i in tg1[0] if i.mark!=""]
hint2_list=get_sentence_ind(audio_path[1],human_result)
tg_sentence1=[tg_sentence1[i].mark for i in hint2_list]
tg_sentence_list=tg_sentence+tg_sentence1
tg_sentence_list.index(human_result.values[0][human_result.columns.get_loc("sentence_test")])
tg_sentence_list



['A BOY FELL FROM A WINDOW',
 'BIG DOGS CAN BE DANGEROUS',
 'THE SHOES WERE VERY DIRTY',
 'THE PLAYER LOST A SHOE',
 "SHE'S DRINKING FROM HER OWN CUP",
 'THE PICTURE CAME FROM A BOOK',
 'THE CAR IS GOING TOO FAST',
 'THE PAINT DRIPPED ON THE GROUND',
 'THE TOWEL FELL ON THE FLOOR',
 'THE KITCHEN WINDOW WAS CLEAN',
 'THE MAILMAN BROUGHT A LETTER',
 'THE MOTHER HEARD THE BABY',
 'SHE FOUND HER PURSE IN THE TRASH',
 "IT'S TIME TO GO TO BED",
 'MOTHER READ THE INSTRUCTIONS',
 'THE DOG IS EATING SOME MEAT',
 'THE PAINTER USES A BRUSH',
 'SWIMMERS CAN HOLD THEIR BREATH',
 "THEY'RE PUSHING AN OLD CAR",
 'THEY HAD TWO EMPTY BOTTLES',
 'THE DOG SLEEPS IN A BASKET',
 "THEY'RE PLAYING IN THE PARK",
 'THE BABY SLEPT ALL NIGHT',
 'THE SALT SHAKER IS EMPTY',
 'THE POLICEMAN KNOWS THE WAY',
 'THE BUCKETS FILL UP QUICKLY',
 'THE JANITOR SWEPT THE FLOOR',
 'THE LADY WASHED THE SHIRT',
 'THE MATCH BOXES ARE EMPTY',
 'THE MAN IS PAINTING A SIGN',
 'THE DOG CAME HOME AT LAST',
 'THEY HEARD A FUNNY NOISE',

In [71]:
audio_path=get_pathset(audio_dir)
talkers=[os.path.basename(i)[10:13] for i in audio_path if "HT1" in i]
talkers=['BRP', 'FAR', 'TUR', 'SPA']

In [72]:
def objective_function(params, sentence_matrix1, human_result, tg_sentence_list, talkers):
    """
    compute the z-value based on our select parameters
    """
    tau, k = params
    similarity_matrix = create_distance_matrix(sentence_matrix1, tau, k)
    out_df=sim_measure(human_result, similarity_matrix,tg_sentence_list,talkers)
    out_df['numIncorrect'] = out_df['numWord'] - out_df['numCorrect']
    out_df['prop_correct'] = out_df['numCorrect'] / out_df['numWord']


    out_df['similarity_scaled'] = (out_df['similarity'] - np.mean(out_df['similarity'])) / (2 * np.std(out_df['similarity']))
    mask_mt = (out_df["Condition"] == "mt") & (out_df["TrainingTalker"].str[-3:] == out_df["TestTalker"])
    mask_st = (out_df["Condition"] == "st ") & (out_df["TrainingTalker"].str[-3:] != out_df["TestTalker"])
    new_df=out_df[mask_mt | mask_st]

    if np.std(new_df['similarity']) < 0.01:
        return {
                    "tau": [tau],
                    "k": [k],
                    "z-value": [0],
                    "mean_sim": [float(np.mean(new_df["similarity"]))],
                    "sd_sim": [float(np.std(new_df["similarity"]))],
                    "error": ["std_sim<0.0001"]  #
                }
    else:
        try:
            import rpy2.robjects as ro
            import rpy2
            from rpy2.robjects import pandas2ri
            pandas2ri.activate()
            base = importr('base')
            stats = importr('stats')
            lme4 = importr('lme4')
            ro.r('options(warn=2)')
            r_data = ro.conversion.py2rpy(new_df)
            #r_data = pandas2ri.py2rpy(new_df)
            formula = Formula('cbind(numCorrect, numIncorrect) ~ 1 + similarity_scaled + (1| TestTalker) + (1|Sentence)')
            glmerControl = lme4.glmerControl(optimizer="bobyqa", optCtrl=ro.vectors.ListVector({'maxfun': 1e6}))
        except Exception as e:
            return {
                    "tau": [tau],
                    "k": [k],
                    "z-value": [0],
                    "mean_sim": [float(np.mean(new_df["similarity"]))],
                    "sd_sim": [float(np.std(new_df["similarity"]))],
                    "error": [e]  #
                }
        try:
            model = lme4.glmer(formula, data=r_data, control=glmerControl, family=stats.binomial(link="logit"))
            #log_likelihood = ro.r['logLik'](model)
            summary = base.summary(model)
            coefficients = summary.rx2('coefficients')
            z_value=coefficients[1][2]
            return {
                    "tau": [tau],
                    "k": [k],
                    "z-value": [z_value],
                    "mean_sim": [float(np.mean(new_df["similarity"]))],
                    "sd_sim": [float(np.std(new_df["similarity"]))],
                    "error": ["NA"]  #
                }
        except Exception as e:
            return {
                    "tau": [tau],
                    "k": [k],
                    "z-value": [0],
                    "mean_sim": [float(np.mean(new_df["similarity"]))],
                    "sd_sim": [float(np.std(new_df["similarity"]))],
                    "error": [str(e)]  #
                }


In [ ]:
#change the tau and k range here:
tau_values = 10**np.arange(-1,1.5,0.25)
k_values = 10**np.arange(-4,2.5,0.5)

#tau_values = 10**np.arange(-1,1,0.25)
#k_values = 10**np.arange(-4,2.2,0.2)

pandas2ri.activate()
params_grid=list(itertools.product(tau_values, k_values))



start_time = time.time()
n_jobs = min(31, multiprocessing.cpu_count())
results = Parallel(n_jobs=n_jobs)(delayed(objective_function)(_, tSNE_representations, human_result, tg_sentence_list, talkers) for _ in params_grid)
end_time = time.time()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.interpolate import griddata

def plot_gridsearch_result(results):
    df = pd.json_normalize(results)
    df['tau'] = df['tau'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['k'] = df['k'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['z-value'] = df['z-value'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['mean_sim'] = df['mean_sim'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['sd_sim'] = df['sd_sim'].apply(lambda x: x[0] if isinstance(x, list) else x)
    tau_vals = tau_values
    k_vals = k_values
    tau_grid, k_grid = np.meshgrid(tau_vals, k_vals)



    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=['z-value', 'mean_sim', 'sd_sim'],
        specs=[[{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}]]
    )


    z_grid = griddata((df['tau'], df['k']), df['z-value'], (tau_grid, k_grid), method='nearest')
    mean_sim_grid = griddata((df['tau'], df['k']), df['mean_sim'], (tau_grid, k_grid), method='nearest')
    sd_sim_grid = griddata((df['tau'], df['k']), df['sd_sim'], (tau_grid, k_grid), method='nearest')


    fig.add_trace(go.Surface(z=z_grid, x=tau_grid, y=k_grid, colorscale='RdYlGn', showscale=False), row=1, col=1)
    fig.add_trace(go.Surface(z=mean_sim_grid, x=tau_grid, y=k_grid, colorscale='RdYlGn', showscale=False), row=1, col=2)
    fig.add_trace(go.Surface(z=sd_sim_grid, x=tau_grid, y=k_grid, colorscale='RdYlGn', showscale=False), row=1, col=3)


    fig.update_layout(
        width=1700,
        height=600,
        title='3D Surface Plots for z-value, mean_sim, and sd_sim from LSR of 24th-Transformer-layer (Bradlow 2023)',
        scene=dict(
            xaxis_title='Tau',
            yaxis_title='K',
            zaxis_title='Z-Value',
            yaxis=dict(
                type='log',
                tickvals=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10],
                ticktext=['1e-4', '1e-3', '1e-2', '1e-1', '1', '10'],
            )
        ),
        scene2=dict(
            xaxis_title='Tau',
            yaxis_title='K',
            zaxis_title='Mean-Sim',
            yaxis=dict(
                type='log',
                tickvals=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10],
                ticktext=['1e-4', '1e-3', '1e-2', '1e-1', '1', '10'],
            )
        ),
        scene3=dict(
            xaxis_title='Tau',
            yaxis_title='K',
            zaxis_title='SD-Sim',
            yaxis=dict(
                type='log',
                tickvals=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10],
                ticktext=['1e-4', '1e-3', '1e-2', '1e-1', '1', '10'],
            ),
            zaxis=dict(
                range=[0,0.5]
            )
        ),
        showlegend=False,
        coloraxis_showscale=False

    )
    fig.show()


In [ ]:
plot_gridsearch_result(results)